<a href="https://www.kaggle.com/code/datascientistsohail/lgbm-se03ep05?scriptVersionId=118285885" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### LGBM Classifier for wine Quality

**Loading the important packages/libraries**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e5/sample_submission.csv
/kaggle/input/playground-series-s3e5/train.csv
/kaggle/input/playground-series-s3e5/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e5/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e5/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e5/sample_submission.csv')

In [3]:
print(df.shape)
print(df_test.shape)

(2056, 13)
(1372, 12)


**Checking for missing or null values**

In [4]:
print('Missing values if any in train set: ', df.isnull().sum().sum())
print('Missing values if any in test set: ', df_test.isnull().sum().sum())

Missing values if any in train set:  0
Missing values if any in test set:  0


In [5]:
useful_features = [c for c in df.columns if c not in ['Id', 'quality']]
print(useful_features)
print('Total number of useful features: ', len(useful_features))

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
Total number of useful features:  11


**Dividing the Dataset into X and y.**

In [6]:
X = df[useful_features]
y = df['quality'].values
X_test = df_test[useful_features]
del df
del df_test

In [7]:
X = X.values
X_test = X_test.values

**Evaluation of the LGBM on Training set**

In [8]:
# Initialize the network, loss function, and optimizer
y_preds = np.zeros(len(X_test))
cohen_kappa_scores = []


# Implement cross-validation
kf = KFold(n_splits=5)
cohen_kappa_scores = []
for train_index, test_index in kf.split(X,y):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    
    model =  LGBMClassifier()
    model.fit(X_train, y_train)
    final_preds = model.predict(X_valid)
    fold_score = cohen_kappa_score(y_valid, final_preds, weights='quadratic')
    cohen_kappa_scores.append(fold_score)
    y_preds += model.predict(X_test)
print("Test Cohen's Kappa Score:", cohen_kappa_scores)

Test Cohen's Kappa Score: [0.4465308532624741, 0.4266534751422211, 0.45769324206524997, 0.42101764863443814, 0.5061938103326843]


**Evaluation of the Pytorch on Test set**

In [9]:
print(y_preds.shape)
print(submission.shape)

(1372,)
(1372, 2)


**Submission**

In [10]:
submission['quality'] = y_preds
submission.to_csv('submission.csv',index = False)